In [1]:
import pandas as pd
import polars as pl
import numpy as np
import scipy.sparse as sp

from tqdm import tqdm
from typing import List, Any
from importlib import reload
import session_recsys as sr

/home/i.nurgaliev/recsys/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## User and items history

In [2]:
data = pl.read_parquet('train.parquet')
data

user_id,artist_id
str,str
"""d705b538-1bd8-…","""69c71d72-7ed8-…"
"""d705b538-1bd8-…","""30bf469f-9abd-…"
"""d705b538-1bd8-…","""a26c9335-2459-…"
"""d705b538-1bd8-…","""69c903b5-dff0-…"
"""d705b538-1bd8-…","""af8eef9d-13aa-…"
"""d705b538-1bd8-…","""293a86ee-6ce7-…"
"""d705b538-1bd8-…","""348f4909-1c48-…"
"""d705b538-1bd8-…","""ad2bf122-726e-…"
"""d705b538-1bd8-…","""cc97fc57-30b5-…"


## Метрики

В этом задании нашей задачей будем оптимизация метрик ndcg@20. Тем не менее, такая метрику сложно интерпретировать и поэтому вам также будет доступно значение метрики hitrate@20, с которой вы уже познакомились в прошлой домашке

В этом датасете идентификаторы представлены в виде строк, но для работы с ними может быть проще сделать преобразование в числа (например, для алгоритмов матричной факторизации)

In [3]:
user_mapping = {k: v for v, k in enumerate(data['user_id'].unique())}
user_mapping_inverse = {k: v for v, k in user_mapping.items()}

artist_mapping = {k: v for v, k in enumerate(data['artist_id'].unique())}
artist_mapping_inverse = {k: v for v, k in artist_mapping.items()}

In [4]:
grouped_df_with_inds = (
    data
    .with_columns([
        pl.col('user_id').apply(user_mapping.get),
        pl.col('artist_id').apply(artist_mapping.get),
    ])
    # для каждого пользователя оставим последние 3 объекта в качестве тестовой выборки,
    # а остальное будем использовать для тренировки
    .groupby('user_id')
    .agg([
        pl.col('artist_id').apply(lambda x: x[:-3]).alias('train_item_ids'),
        pl.col('artist_id').apply(lambda x: x[-3:]).alias('test_item_ids'),
    ])
)

grouped_df_with_inds

user_id,train_item_ids,test_item_ids
i64,list[i64],list[i64]
33184,"[86491, 19721, … 32100]","[10576, 80516, 41209]"
5664,"[88327, 43739, … 785]","[86620, 10576, 60675]"
30128,"[39597, 58878, … 84229]","[1018, 44185, 44346]"
43936,"[84682, 6744, … 60307]","[27088, 55424, 82527]"
49744,"[20047, 585, … 67026]","[31118, 33005, 9885]"
2448,"[71718, 79306, … 17551]","[50014, 4360, 34093]"
43072,"[10241, 74500, … 66018]","[87291, 42596, 24444]"
46752,"[7971, 28536, … 17805]","[86964, 85244, 38926]"
3120,"[29345, 79092, … 34234]","[39692, 86514, 33545]"


In [5]:
median_seq_len = int(grouped_df_with_inds['train_item_ids'].apply(len).median())
print(f"средняя длина сессии {median_seq_len}")

средняя длина сессии 42


In [6]:
reload(sr)
user_item_data = sr.sparse_user_item(grouped_df_with_inds['user_id'],
                                     grouped_df_with_inds['train_item_ids'])

100%|██████████████████████████████████████████████████████████████████████| 50000/50000 [00:04<00:00, 11651.66it/s]


## Бейзлайны

В качестве простого бейзлайна будем рекомендовать самый популярных артистов

Мы хотим сначала провалидировать такое решение, а значит в качестве популярных артистов мы возьмем только тех, кто чаще встречается в `train_item_ids`

In [ ]:
top_artists = (
    grouped_df_with_inds
    .select(pl.col('train_item_ids').alias('artist_id'))
    .explode('artist_id')
    .groupby('artist_id')
    .count()
    .sort('count', descending=True)
    .head(sr.TOP_K + median_seq_len)
)['artist_id'].to_list()

In [ ]:
ndcg_list = []
hitrate_list = []

for user_id, user_history, y_rel in grouped_df_with_inds.rows():
    y_rec = top_artists.copy()
    
    ndcg_list.append(user_ndcg(y_rel, y_rec))
    hitrate_list.append(user_hitrate(y_rel, y_rec))
    
print(f'NDCG@{TOP_K} = {np.mean(ndcg_list):.5f}, Hitrate@{TOP_K} = {np.mean(hitrate_list):.5f}')

Не забываем про фильтрацию просмотренного (для разных доменов и подходов это не всегда улучши рекомендации, но в данном случае дало прирост)

In [ ]:
ndcg_list = []
hitrate_list = []

for user_id, user_history, y_rel in grouped_df_with_inds.rows():
    y_rec = [artist_id for artist_id in top_artists if artist_id not in user_history]
    
    ndcg_list.append(user_ndcg(y_rel, y_rec))
    hitrate_list.append(user_hitrate(y_rel, y_rec))
    
print(f'NDCG@{TOP_K} = {np.mean(ndcg_list):.5f}, Hitrate@{TOP_K} = {np.mean(hitrate_list):.5f}')

## Построим файл с рекомендациями

Для построения рекомендаций теперь можем учесть все возможные данные

In [13]:
top_artists = (
    data
    .groupby('artist_id')
    .count()
    .sort('count', descending=True)
    .head(sr.TOP_K + median_seq_len)
)['artist_id'].to_list()

In [ ]:
submission = []

for user_id, user_history in data.groupby('user_id').agg(pl.col('artist_id')).rows():
    y_rec = top_artists.copy()
    
    submission.append((user_id, y_rec))
    
submission = pl.DataFrame(submission, schema=('user_id', 'y_rec'))
submission.write_parquet('sample_submission.parquet')
submission

Не забывайте, что в файле с рекомендациями должны быть **исходные идентификаторы (строки)**, а не преобразованные в числа!

## Создание рекомендательной системы для авторов музыки Д/З
- В этот раз вы не обладаете знаниями о контенте, зато у вас есть история прослушиваний исполнителей для каждого пользователя.
- Нужно предсказать релевантные рекомендации для каждого пользователя на основе их истории прослушиваний.
- параметры моделей по умолчанию зачастую не являются оптимальными, стоит подобрать наиболее подходящие
- для улучшения метрик стоит обучать модель на всех возможных данных (train + test)
в качестве комбинации методов можно использовать взвешенную сортировку, где вес зависит от позиции в списке рекомендаций и «качества» метода
- При офлайн оценке рекомендуется оценивать ваше решение с помощью валидации по событиям

In [ ]:
reload(sr)
study = sr.optimise_hyperparams(grouped_df_with_inds)
study

### Prepare submission

In [8]:
full_item_ids = (
    data
    .with_columns([
        pl.col('user_id'),
        pl.col('artist_id').apply(artist_mapping.get),
    ])
    .groupby('user_id')
    .agg([
        pl.col('artist_id').alias('history_ids'),
    ])
)

In [10]:
from gensim.models import Word2Vec
# best_params = study.best_params
# best_params = {'sg': 1, 'window': 3, 'ns_exponent': -1.2600306628405324, 'negative': 15, 'min_count': 1, 'vector_size': 16}
best_params = {'sg': 0, 'window': 3, 'ns_exponent': -0.257840485330199, 'negative': 18, 'min_count': 18, 'vector_size': 64}
final_model = Word2Vec(
    full_item_ids['history_ids'].to_list(),
    **best_params,
    hs=0,
    seed=sr.SEED,
    epochs=50
)

In [14]:
reload(sr)
submission = sr.prepare_submission(data, final_model,
                      artist_mapping, artist_mapping_inverse,
                      top_artists)

100%|███████████████████████████████████████████████████████████████████████| 50000/50000 [00:22<00:00, 2234.34it/s]


In [15]:
submission = pl.read_parquet(sr.SUBMITION_FILE)
submission

user_id,y_rec
str,list[str]
"""754ee207-9625-…","[""13145656-b46b-4dba-875f-9b6f7bf5d72e"", ""db2802ef-5aac-4a78-a688-f022944f186b"", … ""ebe601f7-8e0e-429c-b9be-8325337c85a7""]"
"""ad3a1511-a86b-…","[""b4e4b725-d47f-4f79-b044-a14cdd47e980"", ""c8841d2c-613a-40a7-89af-f7201f073c1d"", … ""23c006f8-4187-4a3f-bb23-1c97fc0cd3db""]"
"""c76a0073-673f-…","[""3af88cef-5bff-4ef7-adaf-1f1942408b96"", ""d6e07452-b6f7-4d5e-83ac-5e9413b98cd1"", … ""09f2763a-8edd-4ed6-ba4d-82c73fa4ff36""]"
"""7c0648a0-8e74-…","[""a6cdae3d-16ae-430a-8716-b82f64ed758d"", ""ef87386a-3b2f-45d3-a200-06d5116d5357"", … ""224f17fd-74e2-4d0b-93a7-63bd7af9c01d""]"
"""c2329132-6aca-…","[""0f3684df-3b34-4324-b228-9eb2e53619b8"", ""985d096c-b1fe-473b-972f-f176d7ebd05c"", … ""6d009688-63b4-48d0-ae25-5ab84137f108""]"
"""913e9952-5c4b-…","[""4b83d4b0-9559-4346-ae3a-8eb54632c6f9"", ""b6908c95-ed36-4bc1-bd0a-0ad196a2e387"", … ""e0d0391a-7454-4d3e-a690-950204ef59bf""]"
"""3e545cf7-ebd8-…","[""fbd6e44d-d56a-45d5-bf6d-49ec4bef01b1"", ""c70a9c8c-a44c-4db1-a3ac-268a5326521a"", … ""94b8478a-913b-4560-a633-1cf4b086647a""]"
"""9d2b7fc8-a794-…","[""9f63ba39-eb64-449d-bd6b-be68970c64e3"", ""13145656-b46b-4dba-875f-9b6f7bf5d72e"", … ""f68f9f13-c536-4ff8-ac48-4fe1090a135e""]"
"""617ab812-0bf3-…","[""42cee962-0f50-4728-b887-01cb7a207075"", ""a662889e-d41f-4acd-9712-a983e3c3d91d"", … ""b6a01c95-ca81-4396-861d-35c4e8e659c5""]"
